In [10]:
import json

# Function to read input from file
def read_input_from_file(file_path):
    with open(file_path, 'r') as file:
        input_text = file.read()
    return input_text

# Function to parse sentence tokens and tags
def parse_sentence(sentence):
    tokens_tags = sentence.split('\n')
    tokens = []
    ner_tags = []

    for token_tag in tokens_tags:
        token, tag = token_tag.split()
        tokens.append(token)
        ner_tags.append(tag)

    return tokens, ner_tags

# Main function to process the input text and split as needed
def process_data(input_text):
    sentences = input_text.strip().split('\n\n')  # Split sentences by empty lines
    print(len(sentences))
    output = []

    current_id = 0
    current_sentence = {'id': str(current_id), 'tokens': [], 'ner_tags': []}
    max_length = 500

    for sentence in sentences:
        tokens, ner_tags = parse_sentence(sentence)

        if(len(tokens)>max_length):
            print("large")
            current_sentence = {'id': str(current_id), 'tokens': tokens[:500], 'ner_tags': ner_tags[:500]}
            output.append(current_sentence)
            current_id += 1
            tokens = tokens[500:] 
            ner_tags = ner_tags[500:] 

        current_sentence = {'id': str(current_id), 'tokens': tokens, 'ner_tags': ner_tags}
        output.append(current_sentence)
        current_id += 1


    print(len(output))

    return output

# Function to save output as a JSON file
def save_output_as_json(output, output_file):
    with open(output_file, 'w') as json_file:
        json.dump(output, json_file, indent=4)

# Example usage:
file_path = 'output.txt'  # Replace with the path to your file
output_file = 'output1.json'  # Path where the JSON output will be saved

input_text = read_input_from_file(file_path)
output = process_data(input_text)

# Save output to JSON file
save_output_as_json(output, output_file)

print(f"Output saved to {output_file}")


489
large
large
large
large
large
large
large
large
497
Output saved to output1.json


In [11]:
import json

# 'output.json' dosyasından JSON verisini oku
with open('output.json', 'r') as f:
    json_data = json.load(f)

# Etiketlerin sayısal karşılıkları
label_map = {
    "B-PER": 0, "I-PER": 1,
    "B-OPE": 2, "I-OPE": 3,
    "B-BANK": 4, "I-BANK": 5,
    "B-APP": 6, "I-APP": 7,
    "B-LOC": 8, "I-LOC": 9,
    "B-NET": 10, "I-NET": 11,
    "B-PAY": 12, "I-PAY": 13,
    "B-PKG": 14, "I-PKG": 15,
    "B-LIN": 16, "I-LIN": 17,
    "B-SER": 18, "I-SER": 19,
    "B-OTH": 20, "I-OTH": 21,
    "B-DATE": 22, "I-DATE": 23,
    "B-NUM": 24, "I-NUM": 25,
    "O": 26
}

# Dönüştürme işlevi
def convert_ner_tags(ner_tags):
    return [label_map.get(tag, 26) for tag in ner_tags]

# Ner_tags'ı sayısallaştır
for entry in json_data:
    entry['ner_tags'] = convert_ner_tags(entry['ner_tags'])

# Sayısallaşmış veriyi 'output_number.json' dosyasına yaz
with open('output_number.json', 'w') as f:
    json.dump(json_data, f, indent=4)



In [12]:
import json
from sklearn.model_selection import train_test_split

# data.json dosyasını oku
with open('output_number.json', 'r') as f:
    dataset = json.load(f)

# Burada dataset'in bir liste formatında olduğunu varsayıyoruz
# Eğer JSON dosyasındaki veri farklı bir yapıdaysa buna göre düzenleme yapman gerekebilir

# Eğitim, doğrulama ve test verisini ayır
train_data, temp_data = train_test_split(dataset, test_size=0.2, random_state=42)  # %80 Eğitim, %20 Test ve Doğrulama
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)  # Kalan %10 Test ve %10 Doğrulama

# Bölünmüş veriyi birleştir
final_data = {
    "train": train_data,
    "validation": val_data,
    "test": test_data
}

# Veriyi dataset.json dosyasına yazdır
with open('dataset.json', 'w') as f:
    json.dump(final_data, f, indent=4)
